In [1]:
import numpy as np

def poly_features(x: np.ndarray, degree: int, include_bias: bool = False) -> np.ndarray:
    """
    Génère une matrice de caractéristiques polynomiales à partir d'un vecteur 1‑D.

    Paramètres
    ----------
    x : np.ndarray, shape (n,)
        Données d'entrée.
    degree : int ≥ 1
        Degré polynomial maximal (x¹ … x^degree).
    include_bias : bool, default False
        Si True, ajoute une 1ʳᵉ colonne de 1 (terme biais/constante).

    Retour
    ------
    X_poly : np.ndarray, shape (n, degree + include_bias)
        Matrice des puissances de x, sans boucle Python.
    """
    x = np.asarray(x).reshape(-1)                 # garantit un vecteur 1‑D
    exponents = np.arange(1, degree + 1)          # [1, 2, …, degree]
    X_poly = x[:, None] ** exponents              # broadcasting : (n,1) ** (degree,) -> (n, degree)
    if include_bias:
        X_poly = np.column_stack([np.ones_like(x), X_poly])
    return X_poly

In [2]:
x = np.array([1., 2., 3.])

In [3]:
y = poly_features(x, 3)
print(y)

[[ 1.  1.  1.]
 [ 2.  4.  8.]
 [ 3.  9. 27.]]


In [4]:
y1 = poly_features(x, 3, include_bias=True)
print(y1)

[[ 1.  1.  1.  1.]
 [ 1.  2.  4.  8.]
 [ 1.  3.  9. 27.]]


## Comment ce petit exercice s’inscrit dans la formation
1. Vectorisation & Broadcasting
La logique est exactement la même que dans PyTorch : un tenseur (n,1) multiplié par (degree,) produit (n, degree) sans for‑loop Python.
Savoir raisonner en termes de shape et de broadcast est indispensable quand nous passerons aux tensors PyTorch à la phase 3.
2. Ingénierie de caractéristiques
Avant d’aborder les réseaux de neurones, vous verrez en phase 2 (machine‑learning « classique ») que la régression polynomiale est un bon laboratoire pour parler de sur‑apprentissage et de validation croisée.
Cette fonction est l’équivalent allégé de sklearn.preprocessing.PolynomialFeatures, mais comprendre sa construction vous rend autonome.
3. Performance & bonnes pratiques
Supprimer les boucles Python → 10‑100× plus rapide et libère le GIL ; même philosophie que dans les frameworks DL.
Code compact, lisible et testé : dès la phase 1 nous prenons l’habitude d’écrire des utilitaires réutilisables et de les valider.

In [5]:
import numpy as np
from itertools import combinations_with_replacement

def poly_features(X: np.ndarray, degree: int, include_bias: bool = False) -> np.ndarray:
    """
    Étend un jeu de caractéristiques (n, d_in) en toutes les combinaisons polynomiales
    jusqu'au degré 'degree' inclus, façon scikit‑learn PolynomialFeatures.

    Paramètres
    ----------
    X : np.ndarray, shape (n_samples, d_in)
        Matrice d'entrée.
    degree : int ≥ 1
        Degré polynomial maximal.
    include_bias : bool, default False
        Ajoute une colonne de 1 (terme biais) si True.

    Retour
    ------
    X_poly : np.ndarray, shape (n_samples, d_out)
        Matrice contenant chaque monôme x₁^a · x₂^b · …, pour
        toutes les combinaisons de puissances dont a+b+… ≤ degree.
    """
    X = np.asarray(X, dtype=float)
    if X.ndim == 1:
        X = X[:, None]                      # force (n, 1)

    n_samples, n_features = X.shape

    # 1. Générer les combinaisons d’indices (avec répétitions) jusqu'au degré demandé
    combs = [
        comb
        for deg in range(1, degree + 1)
        for comb in combinations_with_replacement(range(n_features), deg)
    ]

    # 2. Convertir les combinaisons en matrice d'exposants (n_terms, d_in)
    #    ex: pour d_in=3 et comb=(0,2,2)  -> [1,0,2]
    exponents = np.zeros((len(combs), n_features), dtype=int)
    for i, comb in enumerate(combs):
        exponents[i, np.bincount(comb, minlength=n_features).nonzero()] = \
            np.bincount(comb, minlength=n_features)

    # 3. Calcul vectorisé des puissances :
    #      X[..., None]    -> (n_samples, d_in, 1)
    #      exponents.T     -> (d_in, n_terms)
    #      => broadcasting produit (n_samples, d_in, n_terms)
    powered = X[..., None] ** exponents.T

    # 4. Produit sur l'axe des variables pour obtenir chaque monôme
    X_poly = powered.prod(axis=1)           # (n_samples, n_terms)

    if include_bias:
        X_poly = np.column_stack([np.ones((n_samples, 1)), X_poly])

    return X_poly

In [6]:
# Deux variables, degré 2
X = np.array([[1., 2.],
              [3., 4.]])
Xp = poly_features(X, degree=2, include_bias=True)

# Colonnes attendues : [1, x1, x2, x1², x1·x2, x2²]
print(Xp)
# [[1. 1. 2. 1. 2. 4.]
#  [1. 3. 4. 9.12.16.]]

ValueError: shape mismatch: value array of shape (2,) could not be broadcast to indexing result of shape (1,1)